# Trabajo Práctico 1: Análisis Exploratorio de Datos

El dataset seleccionado contiene datos sobre los bateos realizados en la temporada 2020 de MLB (Major League Baseball). El objetivo de este trabajo práctico es realizar un análisis de los datos, con el que más adelante se va a tratar de predecir si una jugada es jonrón (home run) o no.

Se va a trabajar con los archivos train y park_dimensions. Los mismos se van a unir mediante la variable "park".

# 1. Listado de variables y selección

## Detalle de variables

### train.csv
|Variable|Descripción|Comentario|Entrada|Salida|No utilizada
| :- | :- | :- | :-: | :-: | :-:
|bip_id|Identificador único de la pelota en juego.|No es relevante y podría conducir a un overfitting.|||X
|game_date|Fecha del juego.|Formato: YYYY/MM/DD|X||
|home_team|Abreviatura del equipo local.||X||
|away_team|Abreviatura del equipo visitante.||X||
|batter_team|Abreviatura del equipo del bateador.||X||
|batter_name|Nombre del bateador.||X||
|pitcher_name|Nombre del lanzador.||X||
|batter_id|identificador único del bateador.|Podría conducir a un overfitting. Además, se puede utilizar la variable batter_name en su lugar, que brinda más información.|||X
|pitcher_id|identificador único del lanzador.|Podría conducir a un overfitting. Además, se puede utilizar la variable pitcher_name en su lugar, que brinda más información.|||X
|is_batter_lefty|Codificación binaria de bateadores zurdos.|Valores posibles: 0 y 1.|X||
|is_pitcher_lefty|Codificación binaria de lanzadores zurdos.|Valores posibles: 0 y 1.|X||
|bb_type|Clasificación del tipo de bola bateada.|Valores posibles: line_drive, popup, fly_ball, ground_ball|X||
|bearing|Clasificación de la dirección horizontal de la bola que sale del bate.|Valores posibles: left, center, rigth|X||
|pitch_name|Nombre del tipo de lanzamiento.|Valores posibles: Slider, 4-Seam Fastball, Changeup, Curveball, Sinker, Cutter, Knuckle Curve, Split-Finger, Forkball.|X||
|park|Identificador único del estadio.|Podría conducir a un overfitting. Además, se puede utilizar la variable NAME en su lugar, que brinda más información. Hay que aclarar que dicha variable la vamos a usar para hacer un merge de los distintos archivos.|||X
|inning|Número de entrada dentro del juego.||X||
|outs_when_up|Número actual de outs.||X||
|balls|Número actual de bolas.||X||
|strikes|Número actual de strikes.||X||
|plate_x|Posición de la bola con respecto a la placa central.|Se mide en pies, puede ser hacia la izquierda (-) o derecha (+).|X||
|plate_z|Posición de la pelota con respecto al plato de home.|Se mide en pies.|X||
|pitch_mph|Velocidad de la bola lanzada.|Se mide en millas por hora.|X||
|launch_speed|Velocidad de la pelota que sale del bate.|Se mide en millas por hora.|X||
|launch_angle|Ángulo vertical de la pelota que sale del bate.|Se expresan los grados relativos a una horizontal.|X||
|is_home_run|Codificación binaria de jonrones.|Valores posibles: 0 y 1.||X||

### park_dimensions.csv
|Variable|Descripción|Comentario|Entrada|Salida|No utilizada
| :- | :- | :- | :-: | :-: | :-:
|NAME|Nombre del estadio.||X||
|Cover|Indica si el estadio es abierto en la parte superior.|Valores posibles: Dome, Outdoor, Roof|X||
|LF_Dim|Distancia a la pared del jardin izquierdo.|Se mide en pies.|X||
|CF_Dim|Distancia a la pared del campo central.|Se mide en pies.|X||
|RF_Dim|Distancia a la pared del jardin derecho.|Se mide en pies.|X||
|LF_W|Altura de la pared del jardin izquierdo.|Se mide en pies.|X||
|CF_W|Altura de la pared del campo central.|Se mide en pies.|X||
|RF_W|Altura de la pared del jardin derecho.|Se mide en pies.|X||

### Variable de salida
La variable de salida es "is_home_run" que evalúa si una jugada termina en home run. Esto se produce si el bateador golpea la bola con fuerza y logra enviarla a las tribunas o fuera del estadio. Puede tener dos valores:
- 0: No es home run
- 1: Es home run

## Detalle del dataset

In [ ]:
# Importamos las dependencias necesarias.
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns

In [ ]:
# Importamos train.csv y park_dimensions.csv, los unimos utilizando la variable "park"
entrenamiento = pd.read_csv('./train.csv')
estadio = pd.read_csv('./park_dimensions.csv')
completa=entrenamiento.merge(estadio, on="park", how="left")

# Desechamos las variables no utilizadas
completa = completa.drop(['park','bip_id','batter_id','pitcher_id'],axis=1)

In [ ]:
# Tipo de dato de cada columna.
completa.dtypes

In [ ]:
# Dimensiones del dataset.
completa.shape

In [ ]:
# Primeras 5 filas
completa.head()

## 2. Análisis detallado de un conjunto de variables

### Variable de salida y sus implicaciones

In [ ]:
completa.is_home_run.value_counts().plot.bar(title='Número de home runs');

In [ ]:
completa.is_home_run.value_counts().plot.pie(autopct='%1.0f%%',figsize=(5,5), title='Porcentaje de ocurrencias de home run')

Luego de visualizar el gráfico, podemos ver que los valores de la salida no están balanceados, ya que se tienen mas de 40000 (95% aproximadamente) casos en los que no se tiene home run y solo menos de 5000 (5% aproximadamente) donde si se tiene home run. Esta gran diferencia podría traer problemas a la hora de entrenar diferentes modelos, ya que estos podrían optar por responder siempre por el valor que aparece en la mayoría de los casos y en dicho caso no tratarían de predecir. En otras palabras se podrían producir problemas de underfitting.

### Selección de 5 variables de entrada y cómo afectan a la variable de salida.

Las 5 variables de entrada elegidas para dicho punto son:
- launch speed
- launch angle
- pitch_mph
- bearing
- name

#### Variable: launch_speed

In [ ]:
# Distribución de la variable
px.histogram(completa, x='launch_speed', barmode='group', nbins=30)

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='launch_speed', color='is_home_run', barmode='group')

Como conclusión podemos decir que la variable de entrada afecta a la variable de salida ya que cuando alcanza valores entre las 90 y las 110 millas por hora aproximadamente se empiezan a ver ejemplos de home run. Dichos valores describe una distribución simétrica.

#### Variable: launch_angle

In [ ]:
# Distribución de la variable
px.histogram(completa, x='launch_angle', barmode='group', nbins=30)

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='launch_angle', color='is_home_run', barmode='group')

Como conclusión del gráfico anterior podemos ver que los datos de dicha variable de entrada tienen afectan a los da las variables de salida porque podemos ver un rango de valores entre los 18 y 42 grados aproximadamente en los que se produce home run. DIcho rango describe una distribución simétrica de datos.

#### Variable: pitch_mph

In [ ]:
# Distribución de la variable
px.histogram(completa, x='pitch_mph', barmode='group', nbins=30)

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='pitch_mph', color='is_home_run', barmode='group', nbins=30)

En este caso podemos ver que los datos de salida dependen de los datos de la variable de entrada pero en menor medida, ya que tenemos una distribución más uniforme con un rango de valores que va desde el 70 hasta los 100 millas por hora aproximadamente.

#### Variable: bearing

In [ ]:
# Distribución de la variable
completa.bearing.value_counts().plot.pie(autopct='%1.0f%%',figsize=(5,5))

In [ ]:
# Realación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='bearing', color='is_home_run', barmode='group')

Como se aprecia en el gráfico, las variables de entrada afectan a la variable de salida en mayor medida si son del tipo "center", ya que si el bearing es de este tipo tiene mayor probabilidad de ser home run.

#### Variable: NAME

In [ ]:
# Distribución de la variable
completa.NAME.value_counts().plot.pie(autopct='%1.0f%%',figsize=(10,10))

In [ ]:
# Relación entre la variable de entrada seleccionada y la variable de salida
px.histogram(completa, x='NAME', color='is_home_run', barmode='group')

Como se puede ver teniendo en cuenta la distribución y la relación con la variable de salida, podemos decir que la variable de entrada influye poco o casi nada en definir si una jugada es termina en home run o no.

### Variables de entrada elegidas y su transformación

Para utilizar el dataset en un modelo hay muchos cambios que se podrían aplicar. Uno de ellos es **modificar algunos nombres de las variables**, reemplazando las mayúsculas por minúsculas, respetando el formato de escritura. También convertimos la variable "game_date" al tipo correspondiente.

In [ ]:
# Asignamos nuevos nombres a las columnas
renamed_columns = {'NAME': 'name', 'Cover': 'cover', 'LF_Dim': 'lf_dim', 'CF_Dim':'cf_dim',
                   'RF_Dim': 'rf_dim', 'LF_W': 'lf_w', 'CF_W': 'cf_w', 'RF_W': 'rf_w'
                  }
completa.rename(columns=renamed_columns, inplace=True)

# Convertir columna "game_date" de tipo object/string, a datetime
completa['game_date'] = pd.to_datetime(completa['game_date'])

Por otra parte, aplicaremos el **método Scalling** en las variables numéricas, esto ayudará al modelo a converger con mayor rapidez. Se deben escalar las mismas a rangos similares sin alterar su distribución, para ello aplicaremos MaxAbsScaler de sklearn. De esta forma, evitamos que aquellas variables cuyos valores estén en rangos más grandes tengan mayor peso que las demás.


*Variables afectadas:* inning, outs_when_up, balls, strikes, plate_x, plate_z, pitch_mph, launch_speed, launch_angle, lf_dim, cf_dim, rf_dim, lf_w, cf_w, rf_w.

In [ ]:
completa[['inning', 'outs_when_up', 'balls', 'strikes', 'plate_x', 
           'plate_z', 'pitch_mph', 'launch_speed', 'launch_angle', 
           'lf_dim', 'cf_dim', 'rf_dim', 'lf_w', 'cf_w', 'rf_w']
         ].describe(include='all')

En aquellas variables que contienen categorías, decidimos aplicar el método **One-Hot Encoder**, para facilitar el manejo de las mismas por parte de los algoritmos de Machine Learning.

*Variables afectadas:* pitch_name, bb_type, bearing, pitch_name, cover.

Decidimos no transformar aquellas variables binarias o de texto ya que se encuentran correctamente representadas.

*Variables afectadas:* home_team, away_team, batter_team, batter_name, pitcher_nameis_batter_lefty, is_pitcher_lefty.

### Valores nulos y/o extremos

#### Valores nulos

In [ ]:
completa.isnull().sum()

Como se puede ver en la estadistica anterior hay valores nulos en las variables "launch_speed" media, "launch_angle" y "bb_type" eliminar. El tratamiento que se le podría dar a las dos primeras es el de completarlas con la media de obtenida de los otros casos o con los valores de los casos similares. Otra opción podría ser eliminar sus filas ya que se tienen muchos datos y no habría muchos problemas sin se sacan algunos. Con respecto a la variable "bb_type" lo que se puede hacer es eliminar las filas con los valores nulos, ya que son muy pocas.

#### Valores extremos

##### Variable: launch_angle

In [ ]:
# Convertimos en array a la columna launch_angle
launch_angles = np.array(completa.launch_angle)

# Armamos un diagrama de dispersión con la variable launch_angle.
angle_unique, counts = np.unique(launch_angles, return_counts = True)
size = counts * 3
colors = ['blue'] * len(angle_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(angle_unique,np.ones(len(angle_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
# Diagrama de caja de campo launch_angle
completa.boxplot('launch_angle');

##### Variable: launch_speed

In [ ]:
# Convertimos en array a la columna launch_speed
launch_speeds = np.array(completa.launch_speed)

# Armamos un diagrama de dispersión con la variable launch_speed.
speed_unique, counts = np.unique(launch_speeds, return_counts = True)
size = counts * 3
colors = ['blue'] * len(speed_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(speed_unique,np.ones(len(speed_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
# Diagrama de caja de campo launch_speed
completa.boxplot('launch_speed');

##### Variable: plate_x

In [ ]:
# Convertimos en array a la columna plate_x
plate_xs = np.array(completa.plate_x)

# Armamos un diagrama de dispersión con la variable plate_x
platex_unique, counts = np.unique(plate_xs, return_counts = True)
size = counts * 3
colors = ['blue'] * len(platex_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(platex_unique,np.ones(len(platex_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
# Diagrama de caja de campo plate_x
completa.boxplot('plate_x');

In [ ]:
# Convertimos en array a la columna plate_z
plate_zs = np.array(completa.plate_z)

# Armamos un diagrama de dispersión con la variable plate_z
platez_unique, counts = np.unique(plate_zs, return_counts = True)
size = counts * 3
colors = ['blue'] * len(platez_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(platez_unique,np.ones(len(platez_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
# Diagrama de caja de campo plate_z
completa.boxplot('plate_z');

##### Variable: pitch_mph

In [ ]:
pitch_mphs = np.array(completa.pitch_mph)

pitch_mph_unique, counts = np.unique(pitch_mphs, return_counts = True)
size = counts * 3
colors = ['blue'] * len(pitch_mph_unique)
plt.axhline(1,color='k',linestyle='--')
plt.scatter(pitch_mph_unique,np.ones(len(pitch_mph_unique)),s=size,color=colors)
plt.yticks([])
plt.show()

In [ ]:
completa.boxplot('pitch_mph');

De acuerdo a lo visto en los gráficos anteriores, las variables que poseen extremos son "plate_x", "plate_z", "pitch_mph", "launch_speed" y "launch_angle". Lo que se podría hacer con dichos datos es eliminarlos para que no afecten los resultados de los métricas.Cabe aclarar que las variables "lf_dim", "cf_dim", "rf_dim", "lf_w", "cf_w" y "rf_w" también presentan valores extremos, pero en estos casos se deben dejar porque hablan de las dimensiones de los estadios y por lo tanto no se deberían eliminar.

### Existencia de variables altamente correlacionadas con la variable "target"

In [ ]:
# Gráfico de colores de las principales variables
sns.heatmap(completa.corr(), annot=True, cmap='RdYlGn', linewidths=0.2)
fig = plt.gcf()
fig.set_size_inches(20, 8)
plt.show()

Como se ve en el gráfico de colores no existen variables altamente correlacionadas con la variable target, ya que la variables con mayor valor de correlación son "launch_speed" y "launch_angle" con 0,25 y 0,13 respectivamente. Consideramos que esto pasa por la cantidad de variables de entrada del dataset y por el gran desbalance entre los datos de categoría positiva y negativa (si es o no home run).

## 3. Hipótesis sobre los datos

### Hipótesis

#### Hipotésis 1: Hay una mayor tendencia de que la jugada sea home run cuando el ángulo de lanzamiento es mayor o igual a 20° y menor a 40°.

In [ ]:
def condition_h1(row):
    if (row['launch_angle'] >= 20.0) and (row['launch_angle'] < 40.0) and (row['launch_angle'] != 'nan'):
        return 1
    else:
        return 0
    
higher_launch_angle = completa.apply(condition_h1, axis=1)
completa['higher_launch_angle'] = higher_launch_angle

completa[completa.is_home_run==1].higher_launch_angle.value_counts(normalize=True).sort_index().plot.pie(autopct='%1.0f%%', ylabel='Resultado', title=f'% de tiros con ángulo mayor o igual a 20° y menor a 40° que fueron home run')

Según los datos se comprueba que la hipótesis es verdadera porque en el gŕafico anterior se puede observar que hay un 71% de casos en los que se produce home run con el rango especificado de ángulos, frente a un 29% de casos negativos.

#### Hipótesis 2: Hay menos probabilidades de que se produzca un home run si la dirección de la pelota (bearing) es distinto de "rigth" y el tipo de bola es "fly_ball".

In [ ]:
def condition_h2(row):
    if (row['bearing'] != 'rigth') and (row['bb_type'] == 'fly_ball'):
        return 1
    else:
        return 0

center_bearing = completa.apply(condition_h2, axis=1)
completa['center_bearing'] = center_bearing

completa[completa.is_home_run==1].center_bearing.value_counts(normalize=True).sort_index().plot.pie(autopct='%1.0f%%', ylabel='Resultado' , title=f'% de casos donde se cumplen las condiciones y se produce home run')

Según los datos se comprueba que la hipótesis es falsa porque en el gŕafico anterior se puede ver que hay un 55% de casos en los que se produce home run con las condiciones indicadas, frente a un 45% de casos negativos.

#### Hipótesis 3: Si el lanzador es diestro, si la velocidad de la bola es mayor al promedio, si el tipo de bola (bb_type) es "line drive" y  si el tipo de lanzamiento es "Slider", hay una probabilidad de al menos 5% de que se produzca un home run.

In [ ]:
def condition_h3(row):
    if (row['is_pitcher_lefty'] == 0) and (row['bb_type'] == 'fly_ball') and (row['pitch_mph'] < 89) and (row['pitch_name'] == 'Slider'):
        return 1
    else:
        return 0
    
pitcher_lefty_mph = completa.apply(condition_h3, axis=1)
completa['pitcher_lefty_mph'] = pitcher_lefty_mph

completa[completa.is_home_run==1].pitcher_lefty_mph.value_counts(normalize=True).sort_index().plot.pie(ylabel='Resultado', autopct='%1.0f%%', title=f'% de casos donde se cumplen las condiciones y se produce un home run')

Según los datos se comprueba que la hipótesis es verdadera porque en el gŕafico anterior se puede ver que hay un 6% de casos en los que se produce home run con el las condiciones indicadas, frente a un 94% de casos negativos.

### Hallazgos

- Muchas variables, 29 para ser exactos, lo que puede volver más complejo el diseño del modelo de predicción. Como consecuencia a la hora de buscar una función para el modelo, esta va a tener que considerar muchos aspectos.
- Como se pudo observar anteriormente hay un desbalance importante en la variable de salida y muchos casos son parecidos lo que puede afectar al modelo en lo que respecta a la generalización para identificar la mejor predicción.
- Las correlaciones entre las distintas variables con la variable de salida no son tan importantes, por lo que no es muy fácil identificar cual de todas las variables tienen mayor peso. Esto se debe en parte a la gran variedad de casos que se pueden tener en los datos del dataset. Esto podría traer complicaciones a la hora de hacer el modelo y determinar las variables con mayor peso o más relevantes.